In [1]:
import pandas as pd

### Load patent id granted in 2012 and application id in 2017.

In [2]:
grants2012id_df = pd.read_csv('../data/grant2012_all.csv')

In [3]:
grants2012id_df = grants2012id_df.drop("Unnamed: 0", axis=1)

In [4]:
grants2012id_df.head()

,parsed
0,8166569
1,8166570
2,8166571
3,8166572
4,8166573


In [5]:
app2017id_df = pd.read_csv('../data/application2017_all.csv')

In [6]:
app2017id_df = app2017id_df.drop("Unnamed: 0", axis=1)

In [7]:
app2017id_df.head()

,app_id
0,14988955
1,15299629
2,15239362
3,15462276
4,14998443


### (only once) Filter citation of office action which is for 2017 application


citations.csv and office_actions.csv is large. Just filter app_id to reduce size. Do it only once

Below cell need memory more than 15GB.

In [ ]:
citations = pd.read_csv("../data/citations.csv")

In [ ]:
citations.head()

In [ ]:
citations.shape

In [ ]:
citations_2017 = pd.merge(citations, app2017id_df, on="app_id")

In [ ]:
citations_2017.shape

In [ ]:
citations_2017.head()

In [9]:
sum(app2017id_df["app_id"]==12022531)

1

In [24]:
citations_2017.to_pickle("../data/citations_2017.dat")

###  (onlly once) Filer office_actions of app_id is in 2017

In [25]:
office_actions = pd.read_csv("../data/office_actions.csv")

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [26]:
office_actions.shape

(4384532, 23)

In [27]:
office_actions.head()

,app_id,ifw_number,document_cd,mail_dt,art_unit,uspc_class,uspc_subclass,header_missing,fp_missing,rejection_fp_mismatch,...,rejection_103,rejection_112,rejection_dp,objection,allowed_claims,cite102_gt1,cite103_gt3,cite103_eq1,cite103_max,signature_type
0,14150981,100867762,CTFR,2015-10-15,2632,375,219000,0,0,0,...,1,0,0,0,0,0,0,1,2,1
1,14198961,100867788,CTFR,2015-10-15,2699,345,173000,0,0,0,...,1,0,0,0,0,0,0,0,2,1
2,13796589,100867794,CTNF,2015-10-15,3776,606,159000,0,0,0,...,1,0,0,0,0,0,0,0,3,3
3,14673475,100867844,CTNF,2015-10-15,2627,345,175000,0,0,0,...,1,1,1,0,0,0,1,0,4,1
4,14669113,100867850,CTNF,2015-10-15,2666,382,118000,0,0,0,...,0,1,1,0,0,0,0,0,0,1


In [28]:
office_2017 = pd.merge(office_actions, app2017id_df, on="app_id")

In [29]:
office_2017.shape

(26342, 23)

In [30]:
office_2017.to_pickle("../data/office_2017.dat")

### Load citations and office actions which app_id in 2017 only

To reduce footprint, restart kernel here and reload it.

In [8]:
citations_2017 = pd.read_pickle("../data/citations_2017.dat")

In [9]:
office_2017 = pd.read_pickle("../data/office_2017.dat")

### Match office action and citation for 2017 application 2012 grants

In [10]:
citations_2012_2017 = pd.merge(citations_2017, grants2012id_df.astype("str"), on="parsed")

In [11]:
citations_2012_2017.shape, citations_2017.shape

((6745, 9), (278403, 9))

In [102]:
len(set(citations_2012_2017['app_id'])), len(set(citations_2012_2017['parsed']))

(3102, 5424)

## Data setup assumption

Download application of 2017 and grants of 2012, unzip xml, and place under data/

From here: https://bulkdata.uspto.gov/#pats

### Grants 2012

Use "Patent Grant Full Text Data (No Images) (JAN 1976 - PRESENT)".

- Download all zip of https://bulkdata.uspto.gov/data/patent/grant/redbook/fulltext/2012/
   - Use shell script to download zip (https://colab.research.google.com/drive/1gJO0VQ72Xfr_SY2dwWyiTvczIv0_gd4L)
       - save to download.sh and run
- place xml under data/grants2012/
   - ex.data/grants2012/ipg120103.xml data/grants2012/ipg120221.xml
   - Total about 24GB

### Application 2017

Use "Patent Application Full Text Data (No Images) (MAR 15, 2001 - PRESENT)"

- Download all zip of https://bulkdata.uspto.gov/data/patent/application/redbook/fulltext/2017/
   - Use above colab shell script again
- place xml under data/application/2017
    - ex. data/application2017/ipa170105.xml data/appli cation2017/ipa170223.xml
    - Total about 35GB

In [12]:
INDEX_PATH="../data/index"

In [13]:
# currenty lindxr is often updated, so install herer instead of include in Dockerfile

!go get github.com/karino2/lindxr
!go get github.com/karino2/musect

In [14]:
import subprocess

In [15]:
# lindxr index -indexdest index/grants2012 -pattern "<doc-number>" -target "../data/grants2012/ipg12011*.xml"

def build_index(indexdest, pattern, targetpat):
    subprocess.call(["lindxr", "-indexdest", indexdest, "-pattern", pattern, "-target", targetpat])

In [16]:
def subfile(fpath, start, end):
    return subprocess.check_output(["musect", "one", "-start", str(start), "-end", str(end), "-input", fpath],universal_newlines=True).split("\n")



In [17]:
TEMP_SUBFILE_DIR="temp_subfile"

In [18]:
import os
import glob
def _start_end_list_to_file(start_end_list, fpath):
    with open(fpath, "w") as f:
        for start, end in start_end_list:
            f.write("{},{}\n".format(start, end))

REGIONLIST_PATH="{}/regions.txt".format(TEMP_SUBFILE_DIR)
TEMP_OUT_PREFIX="{}/result".format(TEMP_SUBFILE_DIR)
            
def subfiles(fpath, start_end_list):
    os.makedirs(TEMP_SUBFILE_DIR, exist_ok =True)
    _start_end_list_to_file(start_end_list, REGIONLIST_PATH)
    subprocess.call(["musect", "list", "-regions", REGIONLIST_PATH, "-outprefix", TEMP_OUT_PREFIX, "-input", fpath])
    result_files = sorted(glob.glob("{}_*.txt".format(TEMP_OUT_PREFIX)))
    res = []
    for fpath in result_files:
        with open(fpath) as f:
            lines = [line.rstrip("\n") for line in f]
            res.append(lines)
    os.remove(REGIONLIST_PATH)
    [os.remove(path) for path in result_files]
    os.rmdir(TEMP_SUBFILE_DIR)
    return res

### (only once) Create poor man's index 

In [18]:
%%time

build_index(INDEX_PATH+"/grants2012/docnum/", "<doc-number>", "../data/grants2012/ipg*.xml")

CPU times: user 8 ms, sys: 8 ms, total: 16 ms
Wall time: 1min 25s


In [19]:
%%time

build_index(INDEX_PATH+"/grants2012/open_root/", "<us-patent-grant", "../data/grants2012/ipg*.xml")

CPU times: user 8 ms, sys: 8 ms, total: 16 ms
Wall time: 57.7 s


In [20]:
%%time

build_index(INDEX_PATH+"/grants2012/close_root/", "</us-patent-grant>", "../data/grants2012/ipg*.xml")

CPU times: user 4 ms, sys: 12 ms, total: 16 ms
Wall time: 57.4 s


# Load index from files

In [19]:
import glob

In [20]:
indexDNGrants2012 = glob.glob("../data/index/grants2012/docnum/*.idx")

## First load one index from file and filter only real doc-number index

In [21]:
def collect_lines(fpath):
    with open(fpath, "r") as f:
        return [int(l.split(":", 1)[0]) for l in f]

In [22]:
def collect_line_match_tupple(indexfile):
    with open(indexfile, "r") as f:
        return [(int(lnumstr), match) for (lnumstr, match) in (l.rstrip("\n").split(":", 1) for l in f)]

In [25]:
indexDNGrants2012[0]

'../data/index/grants2012/docnum/ipg120501.xml.idx'

In [40]:
fpath = indexDNGrants2012[0]

In [41]:
open_lines = collect_lines(fpath.replace("docnum", "open_root"))
close_lines = collect_lines(fpath.replace("docnum", "close_root"))

In [42]:
len(open_lines), len(close_lines)

(5520, 5520)

In [43]:
tups = collect_line_match_tupple(fpath)

In [44]:
len(tups)

201316

In [45]:
tups[0]

(8, '<doc-number>D0658346</doc-number>')

Retrieve doc-number. Filter non-number value and convert to int here.

In [23]:
import re

In [26]:
DOC_NUM_PAT = re.compile(r'<doc-number>([0-9]+)</doc-number>')

In [27]:
def filer_doc_number_only_number(tups):
    tups_ids = []
    for lnum, docline in tups:
        mat = DOC_NUM_PAT.match(docline)
        if mat:
            tups_ids.append((lnum, int(mat.group(1))))
    return tups_ids


In [46]:
tups_ids = filer_doc_number_only_number(tups)

In [47]:
len(tups_ids), tups_ids[0]

(126788, (16, 29385429))

### Filter doc-number which start from 5 line from &lt;us-patent-grant...&gt;

In [48]:
DOC_NUMBER_OFFSET_FROM_OPEN=5

In [49]:
def search_closest_at(open_lines, line, candidate):
    """
    return maximum index which satisfy open_lines[idx] < line.
    Search is started from candidate.
    """
    for i in range(candidate, len(open_lines)):
        if open_lines[i] > line:
            return i-1
    return len(open_lines)-1


In [30]:
def list_matched_open_idx(open_lines, tups_ids):
    """
    return index of open_lines which tups_ids belong to.
    """
    matched_opens = []
    candidate = 0
    for lnum, id in tups_ids:
        res = search_closest_at(open_lines, lnum, candidate)
        matched_opens.append(res)
        candidate = res
    return matched_opens

In [50]:
matched_opens = list_matched_open_idx(open_lines, tups_ids)

### tupple of (line, doc-number, corresponding idx of open_lines)

In [31]:
def filter_offset_match_only(open_lines, tups_ids, matched_opens, expected_offset):
    return [(lnum, id, matched_opens[idx]) for idx, (lnum, id) in enumerate(tups_ids)
                if lnum == open_lines[matched_opens[idx]]+expected_offset ]

In [51]:
tups_only_5 = filter_offset_match_only(open_lines, tups_ids, matched_opens, DOC_NUMBER_OFFSET_FROM_OPEN)

In [52]:
len(tups_only_5), tups_only_5[0:5]

(4981,
 [(308158, 8166569, 539),
  (309038, 8166570, 540),
  (309428, 8166571, 541),
  (310710, 8166572, 542),
  (311438, 8166573, 543)])

### Check whether how many match to grant2012_all.csv

In [32]:
grants2012id = set(grants2012id_df["parsed"])

In [53]:
sum([id in grants2012id for _, id, _ in tups_only_5])

4981

In [54]:
# seems match to all, but filter for sure.
tups_only_5 = [(lnum, id, openidx) for lnum, id, openidx in tups_only_5 if id in grants2012id]

In [55]:
len(tups_only_5)

4981

### Filter docnumber only in citations_2012_2017

In [34]:
pat_id_set = set(citations_2012_2017["parsed"].astype("int"))

In [49]:
sum(citations_2012_2017["parsed"].astype("int").isna())

0

In [50]:
len(pat_id_set)

5424

In [51]:
final_tups = [(lnum, id, openidx) for lnum, id, openidx in tups_only_5 if id in pat_id_set]

check result

In [52]:
len(final_tups)

98

In [53]:
final_tups[0:5]

[(359894, 8166649, 619),
 (426845, 8166734, 704),
 (445288, 8166761, 731),
 (447021, 8166763, 733),
 (603329, 8166969, 939)]

In [54]:
tups_only_5[0:5]

[(308158, 8166569, 539),
 (309038, 8166570, 540),
 (309428, 8166571, 541),
 (310710, 8166572, 542),
 (311438, 8166573, 543)]

In [55]:
sum(citations_2012_2017["parsed"].astype("int") == 8166649), sum(citations_2012_2017["parsed"].astype("int") == 8166569)

(1, 0)

### Make what I did so far as function

In [35]:
# doc_num_tupples is a list of (line, doc-number, corresponding idx of open_lines)
from collections import namedtuple

DocNumIndex = namedtuple('DocNumIndex', ['open_lines', 'close_lines', "docnum_tupples"])

In [36]:
def one_index_to_docid_tupples(fpath, patidset, docnum_offset=DOC_NUMBER_OFFSET_FROM_OPEN):
    """
    fpath: path for docnum index.
    patidset: Patent id set that we want to keep.
    """
    open_lines = collect_lines(fpath.replace("docnum", "open_root"))
    close_lines = collect_lines(fpath.replace("docnum", "close_root"))
    tups = collect_line_match_tupple(fpath)
    tups_ids = filer_doc_number_only_number(tups)
    matched_opens = list_matched_open_idx(open_lines, tups_ids)
    tups_only_5 = filter_offset_match_only(open_lines, tups_ids, matched_opens, docnum_offset)
    final_tups = [(lnum, id, openidx) for lnum, id, openidx in tups_only_5 if id in patidset]
    return DocNumIndex(open_lines, close_lines, final_tups)


In [39]:
docnum_ind = one_index_to_docid_tupples(fpath, pat_id_set)

In [60]:
len(final_tups), len(docnum_ind.docnum_tupples)

(98, 98)

In [61]:
docnum_ind.docnum_tupples[0:5]

[(359894, 8166649, 619),
 (426845, 8166734, 704),
 (445288, 8166761, 731),
 (447021, 8166763, 733),
 (603329, 8166969, 939)]

In [62]:
# confirm result by checking xml
_, patid, openidx = docnum_ind.docnum_tupples[0]

In [63]:
openidx

619

In [64]:
fpath

'../data/index/grants2012/docnum/ipg120501.xml.idx'

In [65]:
subfile("../data/grants2012/ipg120501.xml",
        docnum_ind.open_lines[openidx]-2, docnum_ind.close_lines[openidx])[0:10]

['<?xml version="1.0" encoding="UTF-8"?>',
 '<!DOCTYPE us-patent-grant SYSTEM "us-patent-grant-v42-2006-08-23.dtd" [ ]>',
 '<us-patent-grant lang="EN" dtd-version="v4.2 2006-08-23" file="US08166649-20120501.XML" status="PRODUCTION" id="us-patent-grant" country="US" date-produced="20120418" date-publ="20120501">',
 '<us-bibliographic-data-grant>',
 '<publication-reference>',
 '<document-id>',
 '<country>US</country>',
 '<doc-number>08166649</doc-number>',
 '<kind>B2</kind>',
 '<date>20120501</date>']

### Now do it for all files under grants2012

In [57]:
import tqdm

In [58]:
def docnum_paths_to_dni_dict(docnum_idx_paths, necessary_docnum_set, docnum_offset=DOC_NUMBER_OFFSET_FROM_OPEN):
    file_dni_dict = {}
    for fpath in tqdm.tqdm(docnum_idx_paths):
        dni = one_index_to_docid_tupples(fpath, necessary_docnum_set, docnum_offset)
        if len(dni.docnum_tupples) == 0:
            print("skip {}".format(fpath))
        else:
            file_dni_dict[fpath] = dni
    return file_dni_dict

In [85]:
pat_id_set = set(citations_2012_2017["parsed"].astype("int"))

In [86]:
file_dni_dict = docnum_paths_to_dni_dict(indexDNGrants2012, pat_id_set, DOC_NUMBER_OFFSET_FROM_OPEN)

100%|██████████| 52/52 [00:38<00:00,  1.36it/s]


In [87]:
len(file_dni_dict.keys())

52

### Is patent id unique? (A. yes).

In [70]:
all_docnum = [patid for fpath in file_dni_dict.keys() for _, patid, _ in file_dni_dict[fpath].docnum_tupples]

In [71]:
len(all_docnum)

5424

In [72]:
len(set(all_docnum))

5424

### Then create dict of {path: list of (patent-id, openlinenum, closelinenum)}


In [59]:
# indexpath_to_xmlpath('../data/index/grants2012/docnum/ipg120821.xml.idx')

def indexpath_to_xmlpath(fpath):
    return fpath.replace("data/index", "data").replace("docnum/", "")[:-4]

In [61]:
def dnidict_to_path_patent_region_tupples(file_dni_dict):
    """
    file_dni_dict: Dict of {doc_num_index_path: DocNumIndex}
    return: Dict of [xmlpath: [list of (doc-number, rootelem-line, closerooteelem-line)]]
    """
    return {indexpath_to_xmlpath(fpath):
                [(patid, dni.open_lines[idx], dni.close_lines[idx]) for _, patid, idx in dni.docnum_tupples ]
                 for fpath, dni in file_dni_dict.items()}

In [88]:
path_to_patregiontup = dnidict_to_path_patent_region_tupples(file_dni_dict)

In [89]:
list(path_to_patregiontup.keys())[0:5]

['../data/grants2012/ipg120821.xml',
 '../data/grants2012/ipg120117.xml',
 '../data/grants2012/ipg120814.xml',
 '../data/grants2012/ipg120731.xml',
 '../data/grants2012/ipg120221.xml']

In [90]:
path_to_patregiontup['../data/grants2012/ipg120313.xml'][0:5]

[(8132283, 253697, 254289),
 (8132340, 291536, 293112),
 (8132342, 293773, 294197),
 (8132376, 331449, 333291),
 (8132468, 422747, 423622)]

### Now collect all patent xml  of what we need!

In [73]:
def collect_all_xml(path_to_patregiontup):
    all_xml_tups = []

    for fpath, tups in tqdm.tqdm(path_to_patregiontup.items()):
        patids = [patid for patid, _, _ in tups]
        startends = [(start-2, end) for _, start, end in tups]
        xmls = ["\n".join(xml) for xml in subfiles(fpath, startends)]
        all_xml_tups.extend(zip(patids, xmls))
    return all_xml_tups

In [91]:
%%time

grants_all_xml_tups = collect_all_xml(path_to_patregiontup)

100%|██████████| 52/52 [06:11<00:00,  7.15s/it]

CPU times: user 3.88 s, sys: 1.65 s, total: 5.53 s
Wall time: 6min 11s


In [92]:
[id for id, _  in grants_all_xml_tups[0:5]]

[8245358, 8245460, 8245733, 8245746, 8245764]

In [93]:
grants_all_df = pd.DataFrame({"parsed": [id for id, _ in grants_all_xml_tups], "xml": [xml for _, xml in grants_all_xml_tups]})

In [94]:
grants_all_df.shape

(5424, 2)

In [95]:
grants_all_df.head()

,parsed,xml
0,8245358,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT..."
1,8245460,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT..."
2,8245733,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT..."
3,8245746,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT..."
4,8245764,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT..."


In [96]:
grants_all_df.to_pickle("../data/grants_2012_from2017_xmldf.dat")

### check result

In [ ]:
grants_all_df = pd.read_pickle("../data/grants_2012_from2017_xmldf.dat")

In [97]:
one_xml = grants_all_df.iloc[1]["xml"].split("\n")

In [98]:
# contains psrsed[1] (8126465) ?
one_xml[0:10]

['<?xml version="1.0" encoding="UTF-8"?>',
 '<!DOCTYPE us-patent-grant SYSTEM "us-patent-grant-v42-2006-08-23.dtd" [ ]>',
 '<us-patent-grant lang="EN" dtd-version="v4.2 2006-08-23" file="US08245460-20120821.XML" status="PRODUCTION" id="us-patent-grant" country="US" date-produced="20120806" date-publ="20120821">',
 '<us-bibliographic-data-grant>',
 '<publication-reference>',
 '<document-id>',
 '<country>US</country>',
 '<doc-number>08245460</doc-number>',
 '<kind>B2</kind>',
 '<date>20120821</date>']

# Collect 2017 applications which are relevant to us

Almost same as grants2012

### (only once) Create poor man's index

In [85]:
%%time

build_index(INDEX_PATH+"/application2017/docnum/", "<doc-number>", "../data/application2017/ipa*.xml")

CPU times: user 28 ms, sys: 24 ms, total: 52 ms
Wall time: 8min 19s


In [86]:
%%time

build_index(INDEX_PATH+"/application2017/open_root/", "<us-patent-application", "../data/application2017/ipa*.xml")

CPU times: user 28 ms, sys: 20 ms, total: 48 ms
Wall time: 7min 48s


In [92]:
%%time

build_index(INDEX_PATH+"/application2017/close_root/", "</us-patent-application>", "../data/application2017/ipa*.xml")

CPU times: user 24 ms, sys: 32 ms, total: 56 ms
Wall time: 7min 42s


### Collect all xml of app2017 with our criteria

In [63]:
indexDNApp2017 = glob.glob("../data/index/application2017/docnum/*.idx")

In [64]:
citations_2012_2017.head()

,app_id,citation_pat_pgpub_id,parsed,ifw_number,action_type,action_subtype,form892,form1449,citation_in_oa
0,12022531,8280486,8280486,NaN,NaN,NaN,1,0,0
1,12242637,8275762,8275762,NaN,NaN,NaN,0,1,0
2,12242637,8108415,8108415,NaN,NaN,NaN,0,1,0
3,12294889,8106092,8106092,NaN,NaN,NaN,0,1,0
4,12489066,8145522,8145522,NaN,NaN,NaN,0,1,0


In [65]:
app_id_set = set(citations_2012_2017["app_id"].astype("int"))
len(app_id_set)

3102

In [66]:
file_dni_dict = docnum_paths_to_dni_dict(indexDNApp2017, app_id_set, docnum_offset=13)

 29%|██▉       | 15/52 [00:04<00:10,  3.49it/s]

skip ../data/index/application2017/docnum/ipa170713.xml.idx


 35%|███▍      | 18/52 [00:04<00:09,  3.63it/s]

skip ../data/index/application2017/docnum/ipa171012.xml.idx


 44%|████▍     | 23/52 [00:06<00:07,  3.65it/s]

skip ../data/index/application2017/docnum/ipa171214.xml.idx


 67%|██████▋   | 35/52 [00:09<00:04,  3.88it/s]

skip ../data/index/application2017/docnum/ipa170406.xml.idx


100%|██████████| 52/52 [00:12<00:00,  4.28it/s]


In [75]:
path_to_patregiontup = dnidict_to_path_patent_region_tupples(file_dni_dict)

In [76]:
%%time

app_all_xml_tups = collect_all_xml(path_to_patregiontup)

100%|██████████| 48/48 [07:45<00:00,  9.69s/it]

CPU times: user 1.34 s, sys: 928 ms, total: 2.27 s
Wall time: 7min 45s


In [77]:
app_all_df = pd.DataFrame({"app_id": [id for id, _ in app_all_xml_tups], "xml": [xml for _, xml in app_all_xml_tups]})

In [78]:
app_all_df.to_pickle("../data/app_2017_by2012_xmldf.dat")

### Check result

In [79]:
app_all_df.shape

(3083, 2)

In [99]:
len(set(app_all_df.app_id))

3077

In [80]:
app_all_df.head()

,app_id,xml
0,14742496,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT..."
1,14348426,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT..."
2,14613336,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT..."
3,14053984,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT..."
4,14590141,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT..."


In [81]:
app_all_df.iloc[1]["xml"].split("\n")[0:20]

['<?xml version="1.0" encoding="UTF-8"?>',
 '<!DOCTYPE us-patent-application SYSTEM "us-patent-application-v44-2014-04-03.dtd" [ ]>',
 '<us-patent-application lang="EN" dtd-version="v4.4 2014-04-03" file="US20170269001A9-20170921.XML" status="PRODUCTION" id="us-patent-application" country="US" date-produced="20170906" date-publ="20170921">',
 '<us-bibliographic-data-application lang="EN" country="US">',
 '<publication-reference>',
 '<document-id>',
 '<country>US</country>',
 '<doc-number>20170269001</doc-number>',
 '<kind>A9</kind>',
 '<date>20170921</date>',
 '</document-id>',
 '</publication-reference>',
 '<application-reference appl-type="utility">',
 '<document-id>',
 '<country>US</country>',
 '<doc-number>14348426</doc-number>',
 '<date>20121001</date>',
 '</document-id>',
 '</application-reference>',
 '<us-application-series-code>14</us-application-series-code>']

### Why not 3102?

In [103]:
not_in = app_id_set- set(app_all_df.app_id)
len(not_in)

25

In [83]:
not_in

{13687937,
 13757778,
 14046229,
 14223855,
 14494367,
 14494415,
 14577580,
 14864955,
 14872276,
 14872315,
 14872716,
 14873160,
 14873292,
 14873845,
 14874591,
 14874615,
 14875024,
 14876632,
 14896485,
 15127826,
 15144598,
 15200023,
 15207409,
 15208151,
 15331960}

In [120]:
# ag did not match 13687937 in all xml. How about app2017.csv?

In [84]:
sum(app2017id_df["app_id"] == 13687937)

1